In [1]:
import pyspark
import findspark
findspark.init()
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SQLContext 
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("SparkProjectProffesorSioutasVonitsanos.com") \
    .getOrCreate()
df = df = spark.read.option("header","true").option("sep", ",").option("multiLine", "true").option("quote","\"").option("escape","\"").option("ignoreTrailingWhiteSpace", True).csv("/Users/evangelosgeraridis/Desktop/decentralized/Monthly_Recycling_and_Waste_Collection_Statistics.csv")
df.show(30)
df.printSchema()
from pyspark.sql.functions import *
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show() # θέλω απλά να δω αν υπάρχουν nan values στο  df 

# θέλουμε η ημερομηνία date να είναι ένα datetype, οπότε τη μετατρέπουμε πιο κάτω , γιατί θέλω να απομονώσω αυτόματα τη χρονοά σε όλο το dataframe
# δηλαδή θέλω για παράδειγμα όπου η ημερομηνία είναι 03/01/2020 για παράδειγμα , εμείς να απομωνόνουμε μόνο τη χρονιά -> μια στήλη που σαν 
# entry εδώ θα έχει το 2020 και αντίστοιχα κάθε χρονιά της καταμέτρησης.

# εδώ επίσης θέλουμε την στήλη TOTAL  (IN TONS) να γίνει αριθμός , integer ουσιαστικά καθώς δε υπάρχουν δεκαδικά και να μην είναι string
# δε το θέλουμε string καθώς δεν θα μπορούμε να κάνουμε πράξεις στα επόμενα ερωτήματα π.χ. όταν θα υπολογιστεί κάποιο sum grouped by κάποια στήλη όπως TYPE
# ή YEAR κ.τ.λ.


# PREPROCESING


from pyspark.sql.types import IntegerType
df = df.withColumn("TOTAL (IN TONS)", df["TOTAL (IN TONS)"].cast(IntegerType()))
from pyspark.sql.types import DateType
from pyspark.sql import functions as F

df = df.withColumn(
            'DATE',
                F.to_date(
                    F.unix_timestamp('DATE', 'MM/dd/yyyy').cast('timestamp')))
df.printSchema()

#ΩΠΡΩΤΟ ΕΡΩΤΗΜΑ

averagedataframe = df.groupBy("TYPE").agg({"TOTAL (IN TONS)":"avg"} ) # to eavala se ena neo df (de xreiazetai)
averagedataframe = averagedataframe.withColumnRenamed('avg(TOTAL (IN TONS))', "AVERAGE OF RECYCLED TYPES")  # κάτι σαν alias , απλά έχω αποθηκεύσει το dataframe
# ως ένα νέο df το averagedataframe , που περιέχει ανά υλικό (groupedby TYPE)  το μέσο όρο ανακυκλωμένου υλικού ( το aggregation agg({"TOTAL (IN TONS)":"avg"})
averagedataframe.show()


# TEΛΟΣ ΠΡΩΤΟΥ ΕΡΩΤΗΜΑΤΟΣ


#ΔΕΥΤΕΡΟ ΕΡΩΤΗΜΑ 


# Για ευκολια και στα επομενα ερωτηματα, απλα θα μετονομασω το TOTAL IN TONS σε TOTAL στο αρχικο νταταφρέιμ

df = df.withColumnRenamed('TOTAL (IN TONS)', "TOTAL")

# ονομαζω data το df που θα χρησιμοποιησω στη συνεχεια για το  ερωτημα 2 
data = df.withColumn('YEAR', year(col('DATE'))) # αφότου κάναμε την στήλη DATE πιο πάνω σε τύπο μεταβήτής datetype, στο ίδιο αρχικό df βάλαμε την στήλη
                                                        # year που περιέχει το χρόνο αντίστοιχα , ο οποίος βγαίνει από την ημερομηνία για να κάνουμε τα επόμενα 
                                                        # ερωτήματα που αφορούν έτη και όχι ημερομηνίες

data = data.groupBy("TYPE" , "YEAR").agg({"TOTAL":"sum"}) # το data πλέον έχει ανά έτος την πληροφορία του συνολικού ποσού (τόνους) 
                                                        # ανακυκλωμένου υλικού και τώρα θα κάνουμε για το query 2


# θα αλλάξουμε το όνομα στήλης sum(TOTAL) στο df data σε SUM για ευκολία στα επόμενα..
# το SUM συμβολίζει το σύνολο ανακυκλώσιμης ποσότητας του κάθε υλικού ανά έτος
data = data.withColumnRenamed('sum(TOTAL)', "SUM")

data.show()
# θα δημιουργήσουμε ένα νέο df βασισμένο στο data , απλά θα περιέχει έξι υλικά της αρεσκείας μας 
df2 = data.filter((data.TYPE == "Curb Garbage") | (data.TYPE == "Scrap Metal") | (data.TYPE == "Misc. Recycling") | (data.TYPE == "Asphalt Debris") | (data.TYPE == "Haz Waste") | (data.TYPE == "E-Waste"))
df2.show()
# # To df2 περιέχει έξι ανακυκλώσιμα υλικά του dataframe data και ανά χρονιά, όπως έχουμε ήδη πει.
# Θα βάλουμε rank με το window partition by (YEAR θέλουμε εμείς εδώ ανά έτος οι ποσότητες των ειδών)
# και το rank 1 θα είναι το μέγιστο (θα μπει με descending order) από αυτά τα 
# έξι υλικά και μετά ένα query για να δούμε που το rank 1 είναι το υλικό που επιλέξαμε

# προσωπικά δε θα πάρω το scrap metal καθώς σε αυτά τα έξι που επέλεξα δεν είναι πουθενά μέγιστο, θα πάρω το curb garbage

# εδώ χρησιμοποιήσαμε το window partition by ανά χρόνο στο dataframe data ( το οποίο βασίζεται στο αρχικό μας df, με τη διαφορά ότι έχουμε απομονώσει τα έτη 
# από την ημερομηνία και έχουμε βρει την συνολική ποσότητα των ανακυκλωμένων υλικών ανά έτος. Με το partition by, βλέπουμε τα στοιχεία 6 ( τόσα επιλέξαμε )
# με φθίνουσα σειρά μεγαλύτερο προς το μικρότερο (αναφορικά πάντα με την ανακυκλωμένη ποσότητα ) ανά έτος partitionBy("YEAR")

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
df2win = Window.partitionBy("YEAR").orderBy(col("SUM").desc())
df2=df2.withColumn("row",row_number().over(df2win)) # βάζουμε νέα στήλη την row (που έχει το rank )

df2.show(40) # εμφανίζω τα πρώτα 40 στοιχεία θα έχει το πρώτο υλικό από τα 6 σε ανακύκλωση ανά έτος δεύτερο ... μέχρι έκτο
# κάποια υλικά λείπουν κάποιες χρονιές (ούτε 0 ούτε nan , λείπει το entry ) φάνηκε και από ιτις διαστάσεις του df , πιθανό λοιπόν να κάποια χρονιά η 
# στήλη rank (1 σημαίνει το μέγιστο σε ανακύκλωση , 2 το αμμέσως μετά κ.τ.λ (descending order) να μην έχει 6 αλλά 5 ή 4 ως το μικρότερο

df2 = df2.filter((col("row") == 1) & (col("TYPE") == 'Curb Garbage'))
df2 = df2.drop('row')
df2.orderBy('YEAR').show()
df2.count()

# κάναμε ένα filter γιατί μας νοιάζει μόνο το μέγιστο στοιχείο ουσιαστικά (col("row") == 1 , δηλαδλη rank = 1 και ταυτόχρονα δηλαδή η στήλη TYPE να έχει
# το στοιχείο curb garbage, δηλαδή θέλω μόνο τα entries όπου ο τύπος του υλικού είναι curb garbage ΚΑΙ ταυτόχρονα να είναι το μέγιστο από τα άλλα 5 
# στοιχεία .
# orderBy year απλά για να φαίνεται πιο ωραίο πρώτα το 11 , 12 μέχρι το  2022 , και drop τη στήλη με τα ranks για να φαίνεται πιο ωραίο
# το count  απλά για να μετρήσει πόσες φορές γίνεται αυτό ( πόσες φορές το Curb Garbage είναι μέγιστο από τα 6 σύνολο) μετράει τις γραμμές του df2
# αλλά το df2 έχει ακριβώς πλέον αυτό που θέλουμε στο ερώτημα 
# αν δε θέλουμε να βλέπουμε και τη ποσότητα ( το sum αθροίσαμε όλες τις ποσότητες του υλικού κάθε καταγεγραμμένη ημερομηνία για να βγει σύνολο στο χρόνο)
# κάνουμε απλά και εδώ ένα drop , εγώ απλά το άφησα
# 12 χρονιές ήταν μέγιστο το curb garbage , δηλαδή κάθε χρονιά από αυτά τα 6 που επιλέχθησαν ήταν το μέγιστο , αν κάνουμε το ίδιο με τα ίδια υλικά 
# για το scrap metal θα δούμε ότι δεν ήταν ποτέ μέγιστο 


# TEΛΟΣ ΔΕΥΤΕΡΟΥ  ΕΡΩΤΗΜΑΤΟΣ



# ΤΡΙΤΟ ΕΡΩΤΗΜΑ

# κάτι παρόμοιο θα κάνουμε και εδώ , απλά με όλα τα υλικά μαζί ουσιαστικά και θα αφήσουμε στο filter μόνο τα ranks <= 5 
# (5 είδη με τις μεγαλύτερες ποσότητες ανακύκλωσης ανά έτος) 

df3win = Window.partitionBy("YEAR").orderBy(col("SUM").desc())
df3=data.withColumn("row",row_number().over(df3win)) # το data είναι το df που θέλουμε ( όλα τα υλικά και συνολική ποσότητα υλικού ανά έτος (SUM))

df3 = df3.filter(col("row") <= 5) # το φίλτρο όπως εξηγήσαμε
df3.orderBy('YEAR').show(60) # όπως πριν γιαμπιο ωραίο και οργανωμένο, δε βάζω το row για να φαίνεται πρώτο δεύτερο.... ανά έτος ξεκινώντας από το 11-22
# # τα top 5 σε ανακύκλωση υλικά ανά έτος και αφού έχουμε χρονιές από το 2011 μέχρι και το 2022 είναι 12 χρόνια επί 5 υλικά (τοπ 5 δείχνει ανά έτος) , άρα 
# έχουμε 60 υλικά  , show(60) το πολύ βασικά , γιατί μπορεί κάποιο να μην έχει entries για έναν ολόκληρο χρόνο κάποιο υλικό

# TEΛΟΣ ΤΡΙΤΟΥ ΕΡΩΤΗΜΑΤΟΣ



# ΤΈΤΑΡΤΟ ΕΡΏΤΗΜΑ 

# ένας ωραίος τρόπος νομίζω που είχα κάνει αρχικά πάλι με window όπως πριν, με partition ανά είδος εδώ καθώς θέλω τη χρονιά όπου το έκαστο είδος 
# έχει max ή min . Το θέμα είναι ότι κάποια υλικά δεν έχουν entries κάποιες χρονιές και το να εκμεταλευτώ domain knowledge σε spark δεν νομίζω ότι είναι καλό


# αρχικά έχουμε στο data το αρχικό df με την χρονιά απομονωμένη σε στήλη και το είχαμε κάνει groupBy και ανά είδος και ανά χρονιά με aggregation
# το άθροισμα των τόνων ανά υλικό και ανά χρονιά , δηλαδή π.χ. curb garbage χ τόνους τη χρονιά y σε σύνολο
# βάση αυτού κάνουμε πάλι ένα partition by TYPE  ( βγαίνει και με ένα απλό groupBy μόνο στο TYPE γιατί θέλουμε ανά είδος το min και το max ,
# αλλά το groupBy βάση μιας στήλης , δε μας εμφανίζει την άλλη ( τη χρονιά εδώ ) και ζητείται και αυτή 


df4win = Window.partitionBy("TYPE").orderBy(col("TYPE").desc()) #orderBy TYPE γιατί αυτό μας αφορά ανά είδος
# βάζουμε σε ένα νεό df που θα έχει μόνο το ελάχιστο ανά είδος κάθε χρονιά , και τοποθετείται σε μια νέα στήλη min (withColumn).
# Το sum είναι το γνωστό όπως στα προηγούμενα σύνολο του κάθε είδους ανά έτος
df4_min = data.withColumn( 'MIN' , min('SUM').over(df4win))
df4_min = df4_min.filter(df4_min.MIN == df4_min.SUM) # φιλτράρισμα πλέον θέλουμε να βλέπουμε μόνο τις γραμμές που το σύνολο του είδους 
# είναι ίσο με το ελάχιστο και για να μην έχω δύο φορές το ίδιο sum ξεφορτώνομαι το sum
# αφήνω και την ποσότητα του ελαχίστου να φαίνεται δεν χρειάζεται καθώς θέλουμε τη χρονιά που έκαστο υλικό έχει το ελάχιστο του
df4_min = df4_min.drop('SUM')
df4_min.show(20)
# συνεπώς βλέπουμε π.χ. το Sidewalk Debris είχε το ελάχιστο του τη χρονιά 2011 με 3900 τόνους ανακύκλωσης εκείνο το έτος.
# το Haz waste έχει πολλές χρονιές , γιατί παραπάνω από μία χρονιά είχαμε 0 ποσότητα αυτού (ανακυκλώσιμη)

# αντίστοιχα σε ένα νέο df4_max για τα μέγιστα

df4_max = data.withColumn( 'MAX' , max('SUM').over(df4win))
df4_max = df4_max.filter(df4_max.MAX == df4_max.SUM)
df4_max = df4_max.drop('SUM')
df4_max.show(50)
# π.χ το Haz Waste είχε μέγιστη ποσότητα ανακύκλωσης το 2014 με  37 τόνους.

# εδώ μπορεί να γίνει join βάσει του είδους αλλά υπάρχει διαφορά στο πλήθος γραμμών ( ένα είδος μπορεί να έχει το ίδιο ελάχιστο πολλές χρονιές ή 
# μέγιστο όπως είδαμε με το Haz Watse , άρα νομίζω καλύτερα όχι 

# TEΛΟΣ ΤΕΤΑΡΤΟΥ ΕΡΩΤΗΜΑΤΟΣ




# ΠΕΜΠΤΟ ΕΡΩΤΗΜΑ 

# το αρχικό df εδώ όχι data , γιατί δεν υπάρχουν μήνες στο data

df5 = df.groupBy('MONTH').agg({"TOTAL":"sum"})
df5 = df5.orderBy(col("sum(TOTAL)").desc())
df5 =  df5.withColumnRenamed('sum(TOTAL)', "TOTAL OF RECYCLED MATERIALS PER MONTHS REGARDLESS OF THE YEAR")
df5.show(5) # τα έχω διατάξει άρα επιλ΄γεω τα πρώτα πέντε όπως ζητείται

# ανεξαρτητως ετους και ειδους , αρα γενικα βλεπουμε οτι ο μηνας Δεκεμβρης σαν συνολο εχει το μεγαλυτερο ποσο σε ανακυκλωσιμα ειδη με 
#  215804 τόνους,  ο μηνας  Μάιος δεύτερος με  177436 τόνους  .... και πέμπτος ο μήνας ο Σεπτέμβριος με 166864 τόνους σύνολο ανακυκλώσιμων ειδών



# TEΛΟΣ ΠΕΜΠΤΟΥ ΕΡΩΤΗΜΑΤΟΣ



# EKTO EΡΩΤΗΜΑ

#Εδώ θέλω αντίστοιχο με το πέμπτο αλλά ανά έτος , δηλαδή το 2020 π.χ. σύνολο από όλα τα έιδη χ ποσότητα

df6 = data.groupBy('YEAR').agg({"SUM":"sum"})
df6 =  df6.withColumnRenamed('sum(SUM)', "TOTAL OF RECYCLED MATERIALS PER YEAR") # αλλάζω όνομα να φαίνεται τι είναι ουσιαστικά 
df6 = df6.orderBy(col("YEAR").desc()) # χρονιά σε φθίνουσα , αχρείαστο
df6.show(12) # 12 χρονιές έχουμε

# TEΛΟΣ ΕΚΤΟΥ ΕΡΩΤΗΜΑΤΟΣ

#Εναλλακτκή διατ΄πωση έκτου ερωτήματος

#Καθώς υπήρξε μια συζήτηση στην αίθουσα για το ερώτημα αυτό, σε περίπτωση που θέλαμε να υπολογίσουμε την συνολική ανακυκλώσιμη ποσότητα του κάθε υλικού
#ανά έτος, την πληροφορία αυτή την έχουμε ήδη αποθηκεύσει σε αυτήν την υλοποίηση στο dataframe data. Είχαμε βρει τη χρονιά και το άθροισμα του υλικού
#κάθε χρονιά αφού πρώτα είχαμε απομονώσει τη στήλη με τα έτη την YEAR. Ουσιαστικά είχαμε εκχωρήσει τότε στο data το ίδιο το dataframe data grouped by TYPE
#και YEAR. Οπότε στην παρούσα φάση αν εκτελέσουμε απλά data.show(144) (το πολύ 144 εγγραφές 12 υλικά επί 12 έτη καταγραφών στο καθένα ):

df77 = data
df77 = df77.withColumnRenamed('SUM' , 'TOTAL OF RECYCLED QUANTITY OF EACH MATERIAL PER YEAR')
df77 = df77.orderBy('YEAR')
#df77 = df77.orderBy('YEAR')
df77.show(144)



# ΕΒΔΟΜΟ ΕΡΩΤΗΜΑ

# κάνουμε το ίδιο ερώτημα ουσιαστικά, απλά αντί για groupBy type και  year βάζουμε μόνο type (στο αρχικό df), εφόσον θέλουμε το aggregation sum( άθροισμα) 
# ανά είδος μόνο ανεξάρτητα του χρόνου που ανακυκλώθηκαν 
# κάνουμε και ένα αχρείαστο orderby για να φαίνονται όλα σε φθίνουσα σειρά από το μέγιστο στο ελάχιστο


df7 = df.groupBy("TYPE").agg({"TOTAL":"sum"})
df7 = df7.withColumnRenamed('sum(TOTAL)', "TOTAL OF RECYCLED TONS OF EACH MATERIAL")
df7 = df7.orderBy(col("TOTAL OF RECYCLED TONS OF EACH MATERIAL").desc())
df7.show(12)

# 12 υλικά 12 entries θα βγουν

# συνεπώς πρώτο σε είδος ανεξαρτήτως έτους σε ανακύκλωση είναι το  Curb Garbage με 1074974 τόνους (συνολικά αθροιστικά και από τα 12 έτη)
# και δωδέλατο, τελευταίο στα 12 αυτά έτη είναι το Haz Waste με  130 τόνους ανακύκλωσης ανά τα έτη όλα

#ΤΕΛΟΣ ΕΒΔΟΜΟΥ ΕΡΩΤΗΜΑΤΟΣ
# ΤΕΛΟΣ PROJECT




22/06/24 19:25:05 WARN Utils: Your hostname, Evangeloss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 150.140.226.242 instead (on interface en0)
22/06/24 19:25:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/24 19:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+--------+---------------+---------------+
|      DATE|   MONTH|           TYPE|TOTAL (IN TONS)|
+----------+--------+---------------+---------------+
|01/31/2011| January| Curb Recycling|            699|
|01/31/2011| January|Misc. Recycling|              0|
|01/31/2011| January|    Bottle Bill|            307|
|01/31/2011| January|    Scrap Metal|              0|
|01/31/2011| January| Recycled Tires|             14|
|01/31/2011| January|     Yard Waste|            269|
|01/31/2011| January| Asphalt Debris|              0|
|01/31/2011| January|Sidewalk Debris|              0|
|01/31/2011| January|      Haz Waste|              0|
|01/31/2011| January|        E-Waste|              0|
|01/31/2011| January|   Curb Garbage|           6816|
|01/31/2011| January|  Misc. Garbage|            287|
|02/28/2011|February| Curb Recycling|            606|
|02/28/2011|February|Misc. Recycling|              0|
|02/28/2011|February|    Bottle Bill|            307|
|02/28/2011|February|    Scr

+---------------+-------------------------+
|           TYPE|AVERAGE OF RECYCLED TYPES|
+---------------+-------------------------+
| Curb Recycling|       1271.1791044776119|
|     Yard Waste|        696.7089552238806|
| Asphalt Debris|        4333.333333333333|
|Misc. Recycling|        309.1641791044776|
| Recycled Tires|       24.044776119402986|
|      Haz Waste|       0.9701492537313433|
|Sidewalk Debris|        393.9166666666667|
|    Bottle Bill|        316.1044776119403|
|        E-Waste|        28.29850746268657|
|    Scrap Metal|       284.85820895522386|
|   Curb Garbage|        8022.194029850746|
|  Misc. Garbage|       1342.4925373134329|
+---------------+-------------------------+

+---------------+----+------+
|           TYPE|YEAR|   SUM|
+---------------+----+------+
|Sidewalk Debris|2014|  5004|
|        E-Waste|2014|   469|
|   Curb Garbage|2013|113856|
|    Scrap Metal|2016|  4056|
|    Scrap Metal|2012|     0|
|        E-Waste|2012|   215|
|        E-Waste|2017|   

+---------------+----+------+---+
|           TYPE|YEAR|   SUM|row|
+---------------+----+------+---+
|   Curb Garbage|2018| 89162|  1|
|Misc. Recycling|2018|  5210|  2|
|    Scrap Metal|2018|  4384|  3|
|        E-Waste|2018|   410|  4|
|      Haz Waste|2018|    20|  5|
|   Curb Garbage|2015| 88936|  1|
|Misc. Recycling|2015|  5158|  2|
|    Scrap Metal|2015|  4142|  3|
|        E-Waste|2015|   428|  4|
|      Haz Waste|2015|    24|  5|
|   Curb Garbage|2022| 11721|  1|
|    Scrap Metal|2022|   664|  2|
|Misc. Recycling|2022|    20|  3|
|        E-Waste|2022|     8|  4|
|      Haz Waste|2022|     0|  5|
|   Curb Garbage|2013|113856|  1|
| Asphalt Debris|2013| 60000|  2|
|    Scrap Metal|2013|  4057|  3|
|        E-Waste|2013|   412|  4|
|Misc. Recycling|2013|    63|  5|
|      Haz Waste|2013|     7|  6|
|   Curb Garbage|2014|113109|  1|
| Asphalt Debris|2014| 60000|  2|
|Misc. Recycling|2014|  4912|  3|
|    Scrap Metal|2014|  4140|  4|
|        E-Waste|2014|   469|  5|
|      Haz Was

+------------+----+------+
|        TYPE|YEAR|   SUM|
+------------+----+------+
|Curb Garbage|2011|102404|
|Curb Garbage|2012| 95045|
|Curb Garbage|2013|113856|
|Curb Garbage|2014|113109|
|Curb Garbage|2015| 88936|
|Curb Garbage|2016| 87325|
|Curb Garbage|2017| 89250|
|Curb Garbage|2018| 89162|
|Curb Garbage|2019| 88479|
|Curb Garbage|2020| 97761|
|Curb Garbage|2021| 97926|
|Curb Garbage|2022| 11721|
+------------+----+------+



+---------------+----+------+---+
|           TYPE|YEAR|   SUM|row|
+---------------+----+------+---+
|   Curb Garbage|2011|102404|  1|
| Asphalt Debris|2011| 28000|  2|
|  Misc. Garbage|2011| 23736|  3|
| Curb Recycling|2011|  8864|  4|
|Sidewalk Debris|2011|  3900|  5|
|   Curb Garbage|2012| 95045|  1|
| Asphalt Debris|2012| 60000|  2|
|  Misc. Garbage|2012| 20948|  3|
| Curb Recycling|2012| 13177|  4|
|Sidewalk Debris|2012|  5000|  5|
|     Yard Waste|2013|  5523|  4|
| Asphalt Debris|2013| 60000|  2|
|Sidewalk Debris|2013|  5004|  5|
| Curb Recycling|2013| 14106|  3|
|   Curb Garbage|2013|113856|  1|
|   Curb Garbage|2014|113109|  1|
| Asphalt Debris|2014| 60000|  2|
| Curb Recycling|2014| 14669|  3|
|     Yard Waste|2014|  5868|  4|
|Sidewalk Debris|2014|  5004|  5|
|   Curb Garbage|2015| 88936|  1|
|  Misc. Garbage|2015| 27042|  2|
| Curb Recycling|2015| 14944|  3|
|     Yard Waste|2015| 11433|  4|
|Misc. Recycling|2015|  5158|  5|
| Curb Recycling|2016| 15472|  3|
|   Curb Garba

+---------------+----+-----+
|           TYPE|YEAR|  MIN|
+---------------+----+-----+
| Curb Recycling|2022| 2314|
|     Yard Waste|2022|  126|
| Asphalt Debris|2011|28000|
|Misc. Recycling|2011|    1|
| Recycled Tires|2022|    9|
|      Haz Waste|2011|    0|
|      Haz Waste|2022|    0|
|      Haz Waste|2020|    0|
|      Haz Waste|2019|    0|
|      Haz Waste|2021|    0|
|Sidewalk Debris|2011| 3900|
|    Bottle Bill|2022|  634|
|        E-Waste|2011|    0|
|   Curb Garbage|2022|11721|
|    Scrap Metal|2012|    0|
|    Scrap Metal|2011|    0|
|  Misc. Garbage|2022|  512|
+---------------+----+-----+

+---------------+----+------+
|           TYPE|YEAR|   MAX|
+---------------+----+------+
| Curb Recycling|2020| 18411|
|     Yard Waste|2019| 15043|
| Asphalt Debris|2013| 60000|
| Asphalt Debris|2014| 60000|
| Asphalt Debris|2012| 60000|
|Misc. Recycling|2019|  5210|
|Misc. Recycling|2020|  5210|
|Misc. Recycling|2021|  5210|
|Misc. Recycling|2016|  5210|
|Misc. Recycling|2017|  5210|


+----+------------------------------------+
|YEAR|TOTAL OF RECYCLED MATERIALS PER YEAR|
+----+------------------------------------+
|2022|                               16008|
|2021|                              153065|
|2020|                              152815|
|2019|                              152221|
|2018|                              149180|
|2017|                              146914|
|2016|                              147764|
|2015|                              156190|
|2014|                              214605|
|2013|                              209352|
|2012|                              203032|
|2011|                              173428|
+----+------------------------------------+

+---------------+---------------------------------------+
|           TYPE|TOTAL OF RECYCLED TONS OF EACH MATERIAL|
+---------------+---------------------------------------+
|   Curb Garbage|                                1074974|
| Asphalt Debris|                                 208000|
|  Mi